# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})
import mlflow
from collections import defaultdict
from scipy.stats import zscore

# original library

import common as com
import pytorch_modeler as modeler
from pytorch_model import DAGMM as Model
from pytorch_utils import filtered_load_model
import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
#with open("./config.yaml", 'rb') as f:
#    config = yaml.load(f)

log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_path = INPUT_ROOT + "/dev_data"
add_dev_path = INPUT_ROOT + "/add_dev_data"
# machine type
MACHINE_TYPE = config['IO_OPTION']['MACHINE_TYPE']
machine_types = os.listdir(dev_path)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
PKL_DIR = OUTPUT_ROOT + '/pkl'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(PKL_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/working/research/dcase2020/result/2D/DAGMM/stable2/v2/config.yaml'

## make path list and train/valid split

In [7]:
############################################################################
# make path set and train/valid split
############################################################################
'''
train_paths[machine_type]['train' or 'valid'] = path
'''
dev_train_paths = {}
add_train_paths = {}
train_paths = {}

In [8]:
for machine_type in machine_types:
    # dev train
    dev_train_paths = ["{}/{}/train/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(dev_path, machine_type))]
    dev_train_paths = sorted(dev_train_paths)
    # add_dev train
    add_train_paths = ["{}/{}/train/".format(add_dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(add_dev_path, machine_type))]
    add_train_paths = sorted(add_train_paths)
    # valid
    dev_valid_paths = ["{}/{}/test/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/test".format(dev_path, machine_type))]
    dev_valid_paths = sorted(dev_valid_paths)
    
    train_paths[machine_type] = {}
    train_paths[machine_type]['train'] = dev_train_paths + add_train_paths
    train_paths[machine_type]['valid'] = dev_valid_paths

## training

In [9]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(train_paths, machine_type)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)         # debug
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    logger.info('TRAINING')
    # parameter setting
    net = Model(sample_rate=config['preprocessing']['sample_rate'],
                window_size=config['preprocessing']['window_size'],
                hop_size=config['preprocessing']['hop_size'],
                mel_bins=config['preprocessing']['mel_bins'],
                fmin=config['preprocessing']['fmin'],
                fmax=config['preprocessing']['fmax'],
                latent_size=config['fit']['latent_size'],
                mixture_size=config['fit']['mixture_size'])
    #pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_PATH'])
    #net = filtered_load_model(net, pretrained_dict)
    optimizer = optim.Adam(net.parameters(), lr=1e-4)
    num_epochs = config['fit']['num_epochs']
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e4, 
                                              max_lr=1e-4, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    criterion = nn.MSELoss()
    try:
        history = modeler.train_net(net, dataloaders_dict, criterion, optimizer, scheduler, num_epochs, writer)
    except:
        history = "hoge"
    # output
    model = history['model']
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    torch.save(model.state_dict(), model_out_path)
    logger.info('\n success:{0} \n'.format(machine_type) + \
                    'model_out_path ==> \n {0}'.format(model_out_path))
    #  close writer for tensorbord
    writer.close()
    #modeler.mlflow_log(history, config, machine_type, model_out_path, tb_log_dir)
    com.toc()
    return history

In [10]:
machine_types

['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']

In [ ]:
for machine_type in machine_types:
    history = run(machine_type)
    #with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    #    pickle.dump(history , file)

2020-11-28 00:19:00,582 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2020-11-28 00:19:00,583 - 00_train.py - INFO - MAKE DATA_LOADER
2020-11-28 00:19:01,742 - 00_train.py - INFO - TRAINING


use: cuda:0


100%|██████████| 29/29 [00:03<00:00,  9.30it/s]
2020-11-28 00:19:38,281 - pytorch_modeler.py - INFO - Epoch 1/300:train_loss:175.318126, tr_rec:37.739856, tr_eng:109.898806, tr_covd:25317.678436, val_AUC:0.511270, val_pAUC:0.497262
100%|██████████| 29/29 [00:03<00:00,  8.65it/s]
2020-11-28 00:20:10,833 - pytorch_modeler.py - INFO - Epoch 2/300:train_loss:119.326634, tr_rec:37.640909, tr_eng:133.325331, tr_covd:13670.638615, val_AUC:0.471741, val_pAUC:0.499409
100%|██████████| 29/29 [00:03<00:00,  9.14it/s]
2020-11-28 00:20:42,565 - pytorch_modeler.py - INFO - Epoch 3/300:train_loss:73.195874, tr_rec:36.902546, tr_eng:180.791487, tr_covd:3642.835772, val_AUC:0.476500, val_pAUC:0.506790
100%|██████████| 29/29 [00:03<00:00,  8.76it/s]
2020-11-28 00:21:14,187 - pytorch_modeler.py - INFO - Epoch 4/300:train_loss:57.997896, tr_rec:34.400811, tr_eng:140.275905, tr_covd:1913.898898, val_AUC:0.484922, val_pAUC:0.506077
100%|██████████| 29/29 [00:02<00:00, 10.10it/s]
2020-11-28 00:21:45,206 - py

100%|██████████| 29/29 [00:03<00:00,  8.58it/s]
2020-11-28 00:34:55,253 - pytorch_modeler.py - INFO - Epoch 30/300:train_loss:-23.314225, tr_rec:1.015323, tr_eng:-296.874783, tr_covd:1071.586203, val_AUC:0.529537, val_pAUC:0.478552
100%|██████████| 29/29 [00:03<00:00,  8.66it/s]
2020-11-28 00:35:26,654 - pytorch_modeler.py - INFO - Epoch 31/300:train_loss:-22.480367, tr_rec:1.005844, tr_eng:-288.162391, tr_covd:1066.005708, val_AUC:0.487917, val_pAUC:0.499034
100%|██████████| 29/29 [00:03<00:00,  8.89it/s]
2020-11-28 00:35:57,717 - pytorch_modeler.py - INFO - Epoch 32/300:train_loss:-25.047729, tr_rec:0.923208, tr_eng:-312.972700, tr_covd:1065.266783, val_AUC:0.540633, val_pAUC:0.504996
100%|██████████| 29/29 [00:03<00:00,  9.20it/s]
2020-11-28 00:36:29,410 - pytorch_modeler.py - INFO - Epoch 33/300:train_loss:-24.153416, tr_rec:1.035445, tr_eng:-305.822896, tr_covd:1078.685819, val_AUC:0.524111, val_pAUC:0.475213
100%|██████████| 29/29 [00:03<00:00,  9.12it/s]
2020-11-28 00:37:00,394 

100%|██████████| 29/29 [00:03<00:00,  8.55it/s]
2020-11-28 00:50:02,066 - pytorch_modeler.py - INFO - Epoch 59/300:train_loss:-33.176009, tr_rec:0.707408, tr_eng:-398.113472, tr_covd:1185.586311, val_AUC:0.500000, val_pAUC:0.500000
100%|██████████| 29/29 [00:03<00:00,  9.07it/s]
2020-11-28 00:50:33,421 - pytorch_modeler.py - INFO - Epoch 60/300:train_loss:-34.582912, tr_rec:0.626219, tr_eng:-411.325750, tr_covd:1184.689004, val_AUC:0.500000, val_pAUC:0.500000
100%|██████████| 29/29 [00:03<00:00,  8.80it/s]
2020-11-28 00:51:03,923 - pytorch_modeler.py - INFO - Epoch 61/300:train_loss:-32.851262, tr_rec:0.419532, tr_eng:-394.181499, tr_covd:1229.471378, val_AUC:0.495723, val_pAUC:0.499775
100%|██████████| 29/29 [00:03<00:00,  8.76it/s]
2020-11-28 00:51:35,268 - pytorch_modeler.py - INFO - Epoch 62/300:train_loss:-32.645801, tr_rec:0.302249, tr_eng:-396.158701, tr_covd:1333.564234, val_AUC:0.512814, val_pAUC:0.500774
100%|██████████| 29/29 [00:03<00:00,  8.75it/s]
2020-11-28 00:52:06,217 

...

100%|██████████| 38/38 [00:04<00:00,  7.84it/s]
2020-11-28 10:06:04,821 - pytorch_modeler.py - INFO - Epoch 291/300:train_loss:-43.831250, tr_rec:0.929768, tr_eng:-514.882920, tr_covd:1345.454967, val_AUC:0.500000, val_pAUC:0.500000
100%|██████████| 38/38 [00:04<00:00,  8.99it/s]
2020-11-28 10:06:39,777 - pytorch_modeler.py - INFO - Epoch 292/300:train_loss:-43.739809, tr_rec:0.931900, tr_eng:-513.422989, tr_covd:1334.118248, val_AUC:0.500000, val_pAUC:0.500000
100%|██████████| 38/38 [00:03<00:00,  9.91it/s]
2020-11-28 10:07:14,473 - pytorch_modeler.py - INFO - Epoch 293/300:train_loss:-43.714876, tr_rec:0.927094, tr_eng:-514.835084, tr_covd:1368.307832, val_AUC:0.500000, val_pAUC:0.500000
100%|██████████| 38/38 [00:04<00:00,  9.41it/s]
2020-11-28 10:07:49,532 - pytorch_modeler.py - INFO - Epoch 294/300:train_loss:-43.671178, tr_rec:0.927770, tr_eng:-514.067451, tr_covd:1361.559642, val_AUC:0.468562, val_pAUC:0.498347
100%|██████████| 38/38 [00:03<00:00,  9.89it/s]
2020-11-28 10:08:24,

elapsed time: 10506.748427629 [sec]


  0%|          | 0/94 [00:00<?, ?it/s]

use: cuda:0


100%|██████████| 54/54 [00:06<00:00,  7.90it/s]
2020-11-28 10:11:53,524 - pytorch_modeler.py - INFO - Epoch 1/300:train_loss:212.926073, tr_rec:34.263457, tr_eng:92.550846, tr_covd:33881.506927, val_AUC:0.498617, val_pAUC:0.498978
  2%|▏         | 2/94 [00:00<00:40,  2.26it/s]

history = run('ToyCar')
with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    pickle.dump(history , file)